In [3]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    "/kaggle/input/speech-txt",  # ✅ your folder name
    glob="**/*.txt",
    loader_cls=TextLoader
)

text_documents = loader.load()
print(text_documents)
print(len(text_documents))


[Document(metadata={'source': '/kaggle/input/speech-txt/speech.txt'}, page_content='Deep in the jungle, the Lion King was taking his day-time sleep under his favorite tree.\n As he was snoring loudly in his sleep, a tiny Mouse came running by. \n The Mouse did not see the sleeping Lion King and by accident ran over his nose! The Lion King woke up mad. \n He grabbed the frightened tiny Mouse with his big paws. The scared Mouse was shaking hard. "Please don\'t eat me!" the Mouse begged. \n "I\'m so small I would barely make a bite. If you let me go free, I promise I will help you one day!" The Lion King laughed very loud. \n "You are too small to ever help me!" But the Mouse looked so afraid that the Lion felt bad for her. "Okay little one, run along then.\n " And the Lion let her go. A few days later, the Lion King was walking through the jungle when he got trapped in a big net! He roared loudly\n  but could not break free. Lucky for him, the tiny Mouse heard his roar and came running v

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20)
documents=text_splitter.split_documents(text_documents)

In [5]:
documents

[Document(metadata={'source': '/kaggle/input/speech-txt/speech.txt'}, page_content='Deep in the jungle, the Lion King was taking his day-time sleep under his favorite tree.'),
 Document(metadata={'source': '/kaggle/input/speech-txt/speech.txt'}, page_content='As he was snoring loudly in his sleep, a tiny Mouse came running by.'),
 Document(metadata={'source': '/kaggle/input/speech-txt/speech.txt'}, page_content='The Mouse did not see the sleeping Lion King and by accident ran over his nose! The Lion King woke'),
 Document(metadata={'source': '/kaggle/input/speech-txt/speech.txt'}, page_content='The Lion King woke up mad.'),
 Document(metadata={'source': '/kaggle/input/speech-txt/speech.txt'}, page_content='He grabbed the frightened tiny Mouse with his big paws. The scared Mouse was shaking hard. "Please'),
 Document(metadata={'source': '/kaggle/input/speech-txt/speech.txt'}, page_content='hard. "Please don\'t eat me!" the Mouse begged.'),
 Document(metadata={'source': '/kaggle/input/sp

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
db = Chroma.from_documents(documents, embeddings)
print("✅ Embeddings ready!")


✅ Embeddings ready!


In [34]:
query = "What did the Lion King learn?"
print(db.similarity_search(query)[0].page_content)


and the Lion King learned that being kind


In [35]:
results = db.similarity_search(query, k=3)
for i, doc in enumerate(results):
    print(f"Result {i+1}:\n", doc.page_content)


Result 1:
 and the Lion King learned that being kind
Result 2:
 The Lion King woke up mad.
Result 3:
 " And the Lion let her go. A few days later, the Lion King was walking through the jungle when he


In [36]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

/tmp/ipykernel_35/2277366689.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="llama2")


Ollama()

In [37]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [39]:
## Chain Introduction
## Create Stuff Docment Chain


from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)